# Visualization and modeling (cont'd)

Python activities to complement material on linear fits in [*Measurements and their Uncertainties*](https://www.oupcanada.com/catalog/9780199566334.html) (*MU*), Chapter 5, "Data visualization and reduction."

* [Preliminaries](#Preliminaries)
* [Linear fits](#Linear-fits)
    * [Explicit formulas for uniform uncertainty](#Explicit-formulas-for-uniform-uncertainty)
        * [Parameter estimates](#Parameter-estimates)
        * [Parameter uncertainties](#Parameter-uncertainties)
    * [Exercise 2](#Exercise-2)
    * [Linear fits with `polyfit`](#Linear-fits-with-polyfit)
        * [Programming notes 1](#Programming-notes-1)
    * [Exercise 3](#Exercise-3)
* [Residual analysis](#Residual-analysis)
    * [Example: Residuals for Ohm's law measurement](#Example&#58;-Residuals-for-Ohm's-law-measurement)
        * [Programming notes 2](#Programming-notes-2)
    * [Exercise 4](#Exercise-4)
* [Summary](#Summary)

## Preliminaries
Before proceeding with this notebook you should review the topics from the [previous notebook](5.0-Visualization-and-modeling.ipynb).

The following code cell includes previously used initialization commands that we will need here. The next cell imports data used in the [previous notebook](5.0-Visualization-and-modeling.ipynb) and redefines some of the variables used there.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Load data
current, voltage, alpha_voltage = np.genfromtxt('data/resistance.csv', skip_header=1, delimiter=',',unpack=True)

# Define separate current vector for model
current_model = np.linspace(0,100)

## Linear fits
When two measurement variables such as $x$ and $y$ satisfy a functional relationship such as

$$ y = f(x;m,c) = mx + c, \label{eq:line}\tag{1}$$

we are typically interested in the values for $m$ and $c$ that offer the best fit to our measurements. Here, "best" means the values of $m$ and $c$ that minimize

$$\chi^2(m,c;\{x_k\},\{y_k\},\{\alpha_k\}) = \sum_{k=1}^N\frac{[y_k - (mx_k +  c)]^2}{\alpha_k^2}, \label{eq:lsq}\tag{2}$$

where each $y_k$ is a noisy measurement at $x_k$ with standard error $\alpha_k$. Note the asymmetry here: all of the  uncertainty is in $y$, and $x$ is assumed to be *known*. Consequently, it is important that you associate the dependent variable with the measurement that has the dominant uncertainty. See *MU* Sec. 9.1 for the case when the uncertainty in $x$ can not be ignored. 

The values $\hat{m}$ and $\hat{c}$ that minimize Eq.&nbsp;(\ref{eq:lsq}) are known as the *least-squares fit parameters* for Eq.&nbsp;(\ref{eq:line}). (Recall that the "hat" notation for $\hat{m}$ and $\hat{c}$ indicates that they are *estimates* of $m$ and $c$, given the data $\{x_k,y_k\}$.) As discussed in *MU*, Eq.&nbsp;(\ref{eq:lsq}) follows from *maximizing the likelihood* that you would obtain your measurements, assuming that they satisfy Eq.&nbsp;(\ref{eq:line}) with a measurement uncertainty $\alpha_k$ for each $y_k$. We can of course consider more general, nonlinear relationships among measurent variables, but linear relationships have some advantages that are worth exploring in depth, as we will see below.

### Explicit formulas for uniform uncertainty

#### Parameter estimates
Let us restrict ourselves to the case of uniform uncertainty for now, so that $\alpha_k = \alpha$ for all $k$. Eq.&nbsp;([2](#mjx-eqn-eq:lsq)) then simplifies to

$$
\chi^2(m,c;\{x_k\},\{y_k\},\alpha) = \frac{1}{\alpha^2}\sum_{k=1}^N[y_k - (mx_k +  c)]^2, \label{eq:lsquniform}\tag{3}
$$

which we can differentiate to find the extremal conditions

\begin{align}
\left.\frac{\partial{\chi^2}}{\partial m}\right|_{\hat{m},\hat{c}} &= -\frac{2}{\alpha^2}\sum_{k=1}^N[y_k - (\hat{m}x_k + \hat{c})]x_k = 0,\\
\left.\frac{\partial{\chi^2}}{\partial c}\right|_{\hat{m},\hat{c}} &= -\frac{2}{\alpha^2}\sum_{k=1}^N[y_k - (\hat{m}x_k + \hat{c})] = 0.
\end{align}

Solving this system of equations for $\hat{m}$ and $\hat{c}$ give *MU* Eqs.&nbsp;(5.1) and (5.2) (rewritten for consistency with the notation here),

\begin{align}
\hat{c} &= \frac{\left(\sum_k x_k^2\right)\left(\sum_k y_k\right) - \left(\sum_k x_k\right)\left(\sum_k x_k y_k\right)}{\Delta}, \label{eq:c-hat}\tag{4}\\
\hat{m} &= \frac{N\sum_k x_k y_k - \left(\sum_k x_k\right)\left(\sum_k y_k\right)}{\Delta},\quad\text{with}\\
\Delta &= N\sum_k x_k^2 - \left(\sum_k x_k\right)^2. \label{eq:m-hat}\tag{5}
\end{align}

These equations express $\hat{m}$ and $\hat{c}$ *as unique, explicit functions of the data* that will *automatically* minimize $\chi^2$ in Eq.&nbsp;(\ref{eq:lsquniform}) for *any* values of $\{x_k, y_k\}$. This is one of the chief advantages of a linear fit: the least-squares fit parameters may be determined unambiguously from the data, without a search algorithm. It is important to recognize, though, that these equations will yield values even when $\{x_k, y_k\}$ do *not* satisfy a linear relationship, so we must always evaluate the quality of the fit before assigning much significance to the best-fit parameters obtained from it.

#### Parameter uncertainties

Now that we have expressed $\hat{m}$ and $\hat{c}$ as functions of $\{x_k, y_k\}$, we can use standard error propagation to compute the uncertainties in $\hat{m}$ and $\hat{c}$. To do this, we must treat each measurement $y_k$ as an independent variable in the functions $\hat{m}$ and $\hat{c}$, all with standard error $\alpha$. Recall that we have assumed the $x_k$ variables as known, so we do not need propagate their uncertainty.

\begin{align}
\alpha_\hat{m}^2 &= \sum_{l = 1}^N\left(\frac{\partial\hat{m}}{\partial y_l}\right)^2\alpha^2 = \frac{\alpha^2}{\Delta^2}\sum_{l = 1}^N\left(Nx_l -  \sum_{k=1}^N x_k\right)^2 = \frac{N}{\Delta}\alpha^2,\\
\alpha_\hat{c}^2 &= \sum_{l = 1}^N\left(\frac{\partial\hat{c}}{\partial y_l}\right)^2\alpha^2 = \frac{\alpha^2}{\Delta^2}\sum_{l = 1}^N\left(\sum_{k=1}^N x_k^2 - x_l\sum_{k=1}^N x_k\right)^2 = \frac{\sum_{k=1}^N x_k^2}{\Delta}\alpha^2.
\end{align}

Taking the square root of these expressions, we have

\begin{align}
\alpha_{\hat{m}} &= \alpha\sqrt{\frac{N}{\Delta}}, & \alpha_{\hat{c}} &= \alpha\sqrt{\frac{\sum_{k=1}^N x_k^2}{\Delta}}, \label{eq:alpha-m-c}\tag{6}
\end{align}

which are equivalent to *MU* Eqs.&nbsp;(5.3) and (5.4) if we substitute $\alpha = \alpha_\text{CU}$, where

$$
\alpha_{CU} = \sqrt{\frac{1}{N-2}\sum_{k=1}^{N}[y_k - (\hat{m}x_k + \hat{c})]^2} \label{eq:alpha-CU}\tag{7}
$$

is known as the *common uncertainty*. The distinction here is that $\alpha$ is assumed to be *known*, just as we assume $\{x_k\}$ known, while we determine $\alpha_\text{CU}$ from the same data that we use to obtain the fit. 

Just as the data will vary from one set of measurements to another, so will $\alpha_\text{CU}$, while $\alpha$ remains fixed.  If your estimate for $\alpha$ is accurate, then the expectation of $\alpha_\text{CU}$ (ie, the average over $N\rightarrow\infty$ data sets) will be

$$
\left\langle \alpha_\text{CU}^2\right\rangle = \alpha^2,
$$

so the difference between $\alpha_\text{CU}$ and $\alpha$ is usually small. But if your estimate for $\alpha$ is *inaccurate*, then the difference between $\alpha_\text{CU}$ and $\alpha$ can be large, so comparing the two provides a good consistency check. The standard way to make this comparison is to compute

$$
\chi^2_\text{min} = \chi^2(\hat{m}, \hat{c}) = \frac{1}{\alpha^2}\sum_{k=1}^N[y_k - (\hat{m}x_k +  \hat{c})]^2 = \frac{\alpha_{CU}^2}{\alpha^2}(N-2),
$$

and check that $\chi^2_\text{min}\approx N - 2$. We compare to $N-2$ here because there are two free fit parameters, $\hat{m}$ and $\hat{c}$—in general, for a fit with $p$ fit parameters, we would compare to $N-p$. For more sophisticated ways to evaluate a fit, see the [Residual analysis](#Residual-analysis) section below.

Now let's compute the linear fit parameters to the current-voltage data in the [Demonstrating Ohm's law](#Example&#58;-Demonstrating-Ohm's-law) example. We compute the parameter uncertainty using both the given uncertainty $\alpha$ and the calculated common uncertainty $\alpha_\text{CU}$; since they are close, you would normally report the values using $\alpha$. You should use $\alpha_\text{CU}$ only if you believe that it provides the best available estimate of your measurement uncertainty. Once we obtain the best-fit parameters, we plot the data (using markers) together with the fit (as a line).

In [ ]:
# Evaluate intermediate expressions
# Number of terms
N = len(current)

# Uniform uncertainty
alpha = alpha_voltage[0]

# Sums over x and y
sx = np.sum(current)
sy = np.sum(voltage)

# Sums over x**2 and x*y
sx2 = np.sum(current**2)
sxy = np.sum(current*voltage)

# Evaluate Delta
Delta = N*sx2 - sx**2

# Put it all together for m_hat and c_hat
c_hat = (sx2*sy - sx*sxy)/Delta
m_hat = (N*sxy - sx*sy)/Delta

# Compute standard error for m_hat and c_hat
alpha_c_hat = alpha*np.sqrt(sx2/Delta)
alpha_m_hat = alpha*np.sqrt(N/Delta)

# Compute chi-squared and common uncertainty for best-fit parameters
chi2fit = np.sum(((voltage - (m_hat*current + c_hat))/alpha)**2)
alpha_CU = alpha*np.sqrt((1/(N-2))*chi2fit)

# Recompute standard error for m_hat and c_hat using common uncertainty
alpha_c_hat_CU = alpha_CU*np.sqrt(sx2/Delta)
alpha_m_hat_CU = alpha_CU*np.sqrt(N/Delta)


# Print results
print("Intercept estimate with given uncertainty: ({0:.0f} ± {1:.0f}) µV".format(1000*c_hat, 1000*alpha_c_hat))
print("Slope estimate with given uncertainty: ({0:.2f} ± {1:.2f}) Ω".format(1000*m_hat, 1000*alpha_m_hat))
print()
print("Intercept estimate with common uncertainty: ({0:.0f} ± {1:.0f}) µV".format(1000*c_hat, 1000*alpha_c_hat_CU))
print("Slope estimate with common uncertainty: ({0:.2f} ± {1:.2f}) Ω".format(1000*m_hat, 1000*alpha_m_hat_CU))
print()
print("Chi-squared for best-fit parameters: {0:.1f}".format(chi2fit))
print("Degrees of freedom (N-2): {0:d}".format(N-2))
print("Given uncertainty: {0:.0f} µV".format(1000*alpha))
print("Common uncertainty: {0:.0f} µV".format(1000*alpha_CU))
print()

# Plot data as markers
plt.plot(current, voltage, 'o')

# Overlay fit as a line
plt.plot(current_model, m_hat*current_model + c_hat, '-')

# Format plot
plt.xlim(0, 100)
plt.ylim(0, 10)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

plt.xlabel('Current ($\mu$A)', fontsize=12)
plt.ylabel('Voltage (mV)', fontsize=12)

plt.legend(['Data', 'Fit'], fontsize=10)

plt.show()

###  Exercise 2
The following cell includes code to import data from the [NIST Statistical Reference Database](https://www.itl.nist.gov/div898/strd/index.html) into the variables `x` and `y` and plots it. The data is taken from a NIST study that involved an ozone sensor calibration; the `y` variable corresponds to the ozone concentration (in nmol/mol) measured by the sensor  of a NIST customer, and the `x` variable corresponds to the measurement with the NIST sensor.  The `y` data corresponds to the first column in the data file, so we list it first in the output of [`genfromtxt`](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html). No measurement uncertainty is provided for this data, so it is appropriate to use the common uncertainty $\alpha_\text{CU}$ as an estimate. Some of the measurements are repeated for the same (or very similar) value of `x`. Use *MU* Eqs. (5.1)–(5.6) to compute the least-squares best-fit parameters $\hat{m}$ and $\hat{c}$ for the model $y=mx+c$, the common uncertainty $\alpha_\text{CU}$, and the uncertainties $\alpha_\hat{m}$ and $\alpha_\hat{c}$ in the parameters.

In [ ]:
# Code cell for Exercise 2
# Use this cell for your response, adding cells if necessary.

# Load data
y, x = np.genfromtxt('https://www.itl.nist.gov/div898/strd/lls/data/LINKS/DATA/Norris.dat', 
                     skip_header=60, unpack=True)

# Plot data
plt.plot(x, y, 'o')
plt.xlabel('NIST sensor')
plt.ylabel('Customer sensor')
plt.show()

### Linear fits with `polyfit`

NumPy includes the [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) function that will fit data with a polynomial model of order $p$,

$$
y = a_0 x^p + a_1 x^{p-1} + \ldots + a_{p-1}x + a_p.
$$

While this is clearly nonlinear in $x$, it is linear in the coefficients $\{a_k\}$, which allows us to determine the best least-squares fit parameters $\{\hat{a}_k\}$ through an analysis similar to the one we developed for a fit to a line.

The code cell below shows how to use [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) to compute $\hat{m}$, $\hat{c}$, $\alpha_\hat{m}$, and $\alpha_\hat{c}$ for the [Demonstrating Ohm's law](#Example&#58;-Demonstrating-Ohm's-law) example.

#### Programming notes 1

As described in the [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) documentation, the first three parameters correspond to the `x` data, the `y` data, and the polynomial degree `deg`. The option `w = 1/alpha_voltage` weights each $y_k - (mx_k + c)$ by $1/\alpha_k$, and the option `cov = 'unscaled'` will cause `polyfit` to use these weights when calculating the parameter uncertainties. To use the common uncertainty to compute the parameter uncertainties, set `cov=True`. Confirm that the results for $\alpha_{\hat{m}}$ and $\alpha_{\hat{c}}$ are the same as those obtained earlier through explicit computation.

In [ ]:
# Fit the data using known parameter uncertainties
p, V = np.polyfit(current, voltage, 1, w=1/alpha_voltage, cov='unscaled')

# Fit the data and use fit residuals to estimate parameter uncertainties
p_scaled, V_scaled = np.polyfit(current, voltage, 1, cov=True)

# Print results
print("Intercept estimate: ({0:.0f} ± {1:.0f}) µV".format(1000*p[1], 1000*np.sqrt(V[1][1])))
print("Slope estimate: ({0:.2f} ± {1:.2f}) Ω".format(1000*p[0], 1000*np.sqrt(V[0][0])))
print("Intercept estimate (scaled): ({0:.0f} ± {1:.0f}) µV".format(1000*p_scaled[1], 1000*np.sqrt(V_scaled[1][1])))
print("Slope estimate (scaled): ({0:.2f} ± {1:.2f}) Ω".format(1000*p_scaled[0], 1000*np.sqrt(V_scaled[0][0])))

###  Exercise 3
The following cell downloads and plots the same data from the [NIST Statistical Reference Database](https://www.itl.nist.gov/div898/strd/index.html) that you used for [Exercise 2](#Exercise-2). Use [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) to compute the least-squares best-fit parameters $\hat{m}$ and $\hat{c}$ and their respective (scaled) uncertainties, $\alpha_\hat{m}$ and $\alpha_\hat{c}$, for the model $y=mx+c$. Assign these values to variable names that are different from the ones you used in [Exercise 2](#Exercise-2), and confirm that the results from [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) are the same as those given in [Exercise 2](#Exercise-2) by explicit computation.

In [ ]:
# Code cell for Exercise 3
# Use this cell for your response, adding cells if necessary.

y, x = np.genfromtxt('https://www.itl.nist.gov/div898/strd/lls/data/LINKS/DATA/Norris.dat', 
                     skip_header=60, unpack=True)
plt.plot(x, y, 'o')
plt.xlabel('NIST sensor')
plt.ylabel('Customer sensor')
plt.show()

## Residual analysis

### Example: Residuals for Ohm's law measurement

The code cell below shows how to add a residual plot to the main plot for the [Demonstrating Ohm's law](#Example&#58;-Demonstrating-Ohm's-law) example. Here are some things to look for in a residual plot.
* The residuals should look random and consistent with a Gaussian distribution.
    * A clear pattern in the residuals is a sign of systematic error that ideally should be fixed in the experiment, but could be accommodated by revising the model function.
    * Outliers often indicate unreliability in your procedure; try to understand how it happened and take another data set if time allows
* The normalized residuals should have a variance close to one.
    * A large discrepancy between the variance of the residuals and your estimated measurement variance is worth revisiting, since it often indicates an error in the estimate of $\alpha$. If you can not identify the source of the discrepancy, you might consider scaling the measurement uncertainy.

#### Programming notes 2

Import the [`GridSpec`](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.gridspec.GridSpec.html) to control the layout of the main plot and the residual plot. The arguments to `GridSpec` create objects `gs` for a grid with 2 rows and 1 column, with a height ratio of 3:1, and with no space between the two $x$-axes. After using [`figure`](https://matplotlib.org/3.1.0/api/figure_api.html) to initiate the figure with the handle `fig`, use the [`add_subplot`](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.figure.Figure.html?highlight=add_subplot#matplotlib.figure.Figure.add_subplot) method to create each plot. Use the [`set_xticklabels`](https://matplotlib.org/3.1.0/api/_as_gen/matplotlib.axes.Axes.set_xticklabels.html?highlight=set_xticklabels) method to remove the tick labels from the main plot—otherwise, they overlap with the residual plot below it.

In [ ]:
from matplotlib.gridspec import GridSpec

# Make a grid with 2 rows, 1 colum, a 3:1 height ratio, and no 
# height space between the rows
gs = GridSpec(2, 1, height_ratios=[3, 1], hspace=0)

# Create figure
fig = plt.figure()
current_range = [0,100]

# Compute the measurement residuals
residuals = voltage - (m_hat*current + c_hat)

# Compute the model curve
current_model = np.linspace(current_range[0], current_range[1])
voltage_model = m_hat*current_model + c_hat

# Make the main plot
ax_main = fig.add_subplot(gs[0])

# Plot data as black dots
plt.errorbar(current, voltage, yerr=alpha, fmt='k.')

# Plot fit as a red line
plt.plot(current_model, voltage_model, 'r-')

# Format top panel: set axis ranges and add y-axis label
# No x-axis label here to avoid overlap with residual plot
plt.xlim(current_range[0], current_range[1])
plt.ylim(0,10)
plt.ylabel("Voltage (mV)")

# Hide x-tick labels to avoid overlap with residual plot
ax_main.set_xticklabels([])

# Make the residual plot with a dotted zero line
# Need to adjust ylim,  yticks, ylabel for readability
ax_res = fig.add_subplot(gs[1])

# Plot residuals as black dots
plt.plot(current, residuals/alpha,'k.')

# Show zero line as a black dotted line
plt.plot(current_range, [0,0], 'k:')

# Format bottom panel: set axis ranges, explicitly set
# y-tick values to avoid overlap with top panel, and
# add labels to x-axis and y-axis
plt.xlim(current_range[0], current_range[1])
plt.ylim(-2, 2)
plt.yticks([-1, 0, 1])
plt.xlabel("Current (µA)")
plt.ylabel("Normalized\nresidual")

# Show plot
plt.show();

### Exercise 4
Following the example above, plot the linear fit from [Exercise 2](#Exercise-2) together with the normalized residuals, using $\alpha_\text{CU}$ as the normalization constant.

In [ ]:
# Code cell for Exercise 4
# Use this cell for your response, adding cells if necessary.



## Summary
Here is a list of what you should be able to do after completing this notebook.

* Explain the significance of $m, c, x_k, y_k$, and $\alpha_k$ in Eq.&nbsp;(2).
* Explain how Eq.&nbsp;(2) is used to fit a theoretical model to experimental data.
* Use Eqs.&nbsp;(4)–(7) to compute the parameters and parameter uncertainties of a linear fit when the experimental uncertainty is uniform.
* Compute $\chi^2_\mathrm{min}$ for a linear fit and use it to evaluate the fit quality.
* Use [`polyfit`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyfit.html) to compute the parameters and parameter uncertainties of a linear fit when the experimental uncertainty is uniform.
* Use residual analysis to evaluate the quality of a fit.
* Create a figure that includes experimental data with errorbars and a theoretical fit to the data in one panel, and a residual plot in a second panel.

##### About this notebook
Notebook by J. S. Dodge, 2020. The notebook text is licensed under the Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License. See more at [Creative Commons](http://creativecommons.org/licenses/by-nc-nd/4.0/). The notebook code is open source under the [MIT License](https://opensource.org/licenses/MIT).